# Analyzing Coronovirus incidents across the globe - Data Science Portfolio Project

# Data import and sanitization

### Import libraries 

In [21]:
# Import libraries
import pandas as pd

## Load and Inspect Datasets

We will corelate and analyze data between two csv files:

1) coronavirus_report.csv
2) population_by_country.csv

In [22]:
# Load coronovirus report dataset

incidents = pd.read_csv('coronavirus_report.csv')
print(incidents.info())
incidents.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248346 entries, 0 to 248345
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   date             248346 non-null  object 
 1   location         248346 non-null  object 
 2   new_cases        248075 non-null  float64
 3   new_deaths       228508 non-null  float64
 4   total_cases      248337 non-null  float64
 5   total_deaths     228592 non-null  float64
 6   weekly_cases     246871 non-null  float64
 7   weekly_deaths    227322 non-null  float64
 8   biweekly_cases   245118 non-null  float64
 9   biweekly_deaths  225553 non-null  float64
dtypes: float64(8), object(2)
memory usage: 18.9+ MB
None


,date,location,new_cases,new_deaths,total_cases,total_deaths,weekly_cases,weekly_deaths,biweekly_cases,biweekly_deaths
0,2020-02-24,Afghanistan,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
1,2020-02-25,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
2,2020-02-26,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
3,2020-02-27,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
4,2020-02-28,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN


Some initial observations:

1. During the coronavirus outbreak, we recorded a total of 248,346 cases and deaths.
2. However, it appears that many columns contain a significant number of missing values.

In [3]:
# Load population by country dataset

population = pd.read_csv('population_by_country.csv')
print(population.info())
population.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16400 entries, 0 to 16399
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country_code  16400 non-null  object 
 1   country_name  16400 non-null  object 
 2   year          16400 non-null  int64  
 3   value         16400 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 512.6+ KB
None


,country_code,country_name,year,value
0,ABW,Aruba,1960,54608.0
1,ABW,Aruba,1961,55811.0
2,ABW,Aruba,1962,56682.0
3,ABW,Aruba,1963,57475.0
4,ABW,Aruba,1964,58178.0


Some initial observations:

1. We have 16400 entries

2. It appears there are no missing values in any of the columns

3. The column names and data types look correct

## Data Cleaning + Preparation 

Before we can merge incidents and population into a single DataFrame, we'll need to clean the column names and row values in both DataFrames.

We want to make sure that all column names are lower case and have no hidden whitespace. We also want to make sure that there is no hidden whitespace in any of the text data, since we will be merging on those columns.

Let's save some time by creating a function that can be applied to all DataFrames in our analysis.

In [4]:
def clean_df_text(df):
    # clean column names
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.strip()

    # clean row values of object columns
    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = df[column].str.title()
            df[column] = df[column].str.strip()

    return df

In [5]:
# Clean incidents
clean_incidents = clean_df_text(incidents)
clean_incidents.head()

,date,location,new_cases,new_deaths,total_cases,total_deaths,weekly_cases,weekly_deaths,biweekly_cases,biweekly_deaths
0,2020-02-24,Afghanistan,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
1,2020-02-25,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
2,2020-02-26,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
3,2020-02-27,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
4,2020-02-28,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN


In [6]:
# Clean population
clean_population = clean_df_text(population)
clean_population.head()

,country_code,country_name,year,value
0,Abw,Aruba,1960,54608.0
1,Abw,Aruba,1961,55811.0
2,Abw,Aruba,1962,56682.0
3,Abw,Aruba,1963,57475.0
4,Abw,Aruba,1964,58178.0


Now we want to uppercase all country three letter codes.

In [41]:
def uppercase_country_codes(df, column_name):
    df[column_name] = df[column_name].str.upper()
    return df

clean_population = clean_population.copy()
clean_population = uppercase_country_codes(clean_population, 'country_code')
clean_population.head()

,country_code,country_name,year,value
0,ABW,Aruba,1960,54608.0
1,ABW,Aruba,1961,55811.0
2,ABW,Aruba,1962,56682.0
3,ABW,Aruba,1963,57475.0
4,ABW,Aruba,1964,58178.0


Let's rename location row to country_name on clean_incidents dataframe.

In [42]:
column_mapper = {'location':'country_name'}
clean_incidents = clean_incidents.rename(mapper=column_mapper, axis=1)
clean_incidents.head()

,date,country_name,new_cases,new_deaths,total_cases,total_deaths,weekly_cases,weekly_deaths,biweekly_cases,biweekly_deaths
0,2020-02-24,Afghanistan,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
1,2020-02-25,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
2,2020-02-26,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
3,2020-02-27,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
4,2020-02-28,Afghanistan,0.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN


Let's filter clean_population table to get the total population per country for 2021.

In [28]:
population_2021 = clean_population[clean_population['year'] == 2021]
print(population_2021)

      country_code                 country_name  year        value
61             ABW                        Aruba  2021     106537.0
123            AFE  Africa Eastern And Southern  2021  702976832.0
185            AFG                  Afghanistan  2021   40099462.0
247            AFW   Africa Western And Central  2021  478185907.0
309            AGO                       Angola  2021   34503774.0
...            ...                          ...   ...          ...
16151          XKX                       Kosovo  2021    1786038.0
16213          YEM                  Yemen, Rep.  2021   32981641.0
16275          ZAF                 South Africa  2021   59392255.0
16337          ZMB                       Zambia  2021   19473125.0
16399          ZWE                     Zimbabwe  2021   15993524.0

[265 rows x 4 columns]


Now let's merge these two tables 

In [29]:
merged_data = pd.merge(left=clean_incidents, right=population_2021, on='country_name', how='right')
print(merged_data.head())

         date country_name  new_cases  new_deaths  total_cases  total_deaths  \
0  2020-03-13        Aruba        2.0         NaN          2.0           NaN   
1  2020-03-14        Aruba        0.0         NaN          2.0           NaN   
2  2020-03-15        Aruba        0.0         NaN          2.0           NaN   
3  2020-03-16        Aruba        0.0         NaN          2.0           NaN   
4  2020-03-17        Aruba        1.0         NaN          3.0           NaN   

   weekly_cases  weekly_deaths  biweekly_cases  biweekly_deaths country_code  \
0           NaN            NaN             NaN              NaN          ABW   
1           NaN            NaN             NaN              NaN          ABW   
2           NaN            NaN             NaN              NaN          ABW   
3           NaN            NaN             NaN              NaN          ABW   
4           NaN            NaN             NaN              NaN          ABW   

   year     value  
0  2021  106537.0 

Now, let's clean up the DataFrame by removing unnecessary columns and renaming the 'value' column to 'population_2021' for better clarity in our analysis.

In [45]:
drop_columns = ['year', 'weekly_cases', 'weekly_deaths', 'biweekly_cases', 'biweekly_deaths']
clean_merged_data = merged_data.drop(labels=drop_columns, axis=1)

In [46]:
column_mapper = {'value': 'population_2021'}
new_data = clean_merged_data.rename(mapper=column_mapper, axis=1)
print(new_data.sort_values(by='total_deaths', ascending=False))

              date country_name  new_cases  new_deaths  total_cases  \
197808  2023-03-09        World   177325.0      1319.0  676467356.0   
197807  2023-03-08        World   179384.0      1445.0  676290031.0   
197806  2023-03-07        World   130437.0       923.0  676110647.0   
197805  2023-03-06        World    63196.0       366.0  675980210.0   
197804  2023-03-05        World    59988.0       186.0  675922170.0   
...            ...          ...        ...         ...          ...   
200856  2020-03-31       Zambia        0.0         NaN         35.0   
200857  2020-04-01       Zambia        1.0         NaN         36.0   
201930  2020-03-20     Zimbabwe        1.0         NaN          1.0   
201931  2020-03-21     Zimbabwe        2.0         NaN          3.0   
201932  2020-03-22     Zimbabwe        0.0         NaN          3.0   

        total_deaths country_code  population_2021  
197808     6881575.0          WLD     7.888409e+09  
197807     6880256.0          WLD     7.8

We need to clean up some invalid country names and identify which countries were most affected by the coronavirus.

In [47]:
# we want to filter out some non valid country names
exclude_names = ['World', 'High Income', 'Upper Middle Income', 'Lower Middle Income', 'European Union', 'North America']

# Filter out the aggregate names early
cases = new_data[~new_data['country_name'].isin(exclude_names)]

# Aggregate the data to find the maximum number of total cases and deaths for each country
country_impact = cases.groupby('country_name').max()[['total_cases', 'total_deaths', 'population_2021']]

# Sort the data to find the countries with the highest number of total cases
most_affected_by_cases = country_impact.sort_values(by='total_cases', ascending=False).drop(labels='total_deaths', axis=1)

# Sort the data to find the countries with the highest number of total deaths
most_affected_by_deaths = country_impact.sort_values(by='total_deaths', ascending=False).drop(labels='total_cases', axis=1)

print("Most affected by cases:")
print(most_affected_by_cases.head(10))
print("\nMost affected by deaths:")
print(most_affected_by_deaths.head(10))

Most affected by cases:
                total_cases  population_2021
country_name                                
United States   103802702.0     3.318937e+08
India            44690738.0     1.407564e+09
France           39701767.0     6.774963e+07
Germany          38249060.0     8.319608e+07
Brazil           37081209.0     2.143262e+08
Japan            33320438.0     1.256816e+08
Italy            25603510.0     5.910967e+07
United Kingdom   24425313.0     6.732657e+07
Spain            13770429.0     4.741575e+07
Vietnam          11526994.0     9.746803e+07

Most affected by deaths:
                total_deaths  population_2021
country_name                                 
United States      1123836.0     3.318937e+08
Brazil              699276.0     2.143262e+08
India               530779.0     1.407564e+09
Mexico              333188.0     1.267051e+08
United Kingdom      219948.0     6.732657e+07
Peru                219539.0     3.371547e+07
Italy               188322.0     5.910967e

Quick observations:

1. The United States was the most affected country in terms of both case numbers and death rates.
2. The top three countries most affected by coronavirus cases are the United States, India, and France.
3. The top three countries most affected by coronavirus deaths are the United States, Brazil, and India.

## Next, we aim to estimate the ratios of total cases to population and total deaths to population for each country.

For simplicity, we assume that the 2021 population remains stable throughout the three-year period of the coronavirus pandemic.

In [48]:
most_affected_by_cases['cases/population %'] = round((most_affected_by_cases['total_cases'] / most_affected_by_cases['population_2021']) * 100, 2)

# Calculating the percentage of the population affected by total deaths
most_affected_by_deaths['deaths/population %'] = round((most_affected_by_deaths['total_deaths'] / most_affected_by_deaths['population_2021']) * 100, 3)

# Display the updated DataFrame with the new columns
print("Most affected by cases ratios:")
print(most_affected_by_cases.head(10))
print("\nMost affected by deaths ratios:")
print(most_affected_by_deaths.head(10))

Most affected by cases ratios:
                total_cases  population_2021  cases/population %
country_name                                                    
United States   103802702.0     3.318937e+08               31.28
India            44690738.0     1.407564e+09                3.18
France           39701767.0     6.774963e+07               58.60
Germany          38249060.0     8.319608e+07               45.97
Brazil           37081209.0     2.143262e+08               17.30
Japan            33320438.0     1.256816e+08               26.51
Italy            25603510.0     5.910967e+07               43.32
United Kingdom   24425313.0     6.732657e+07               36.28
Spain            13770429.0     4.741575e+07               29.04
Vietnam          11526994.0     9.746803e+07               11.83

Most affected by deaths ratios:
                total_deaths  population_2021  deaths/population %
country_name                                                      
United States      112

In [49]:
print(cases.head(15))

          date country_name  new_cases  new_deaths  total_cases  total_deaths  \
0   2020-03-13        Aruba        2.0         NaN          2.0           NaN   
1   2020-03-14        Aruba        0.0         NaN          2.0           NaN   
2   2020-03-15        Aruba        0.0         NaN          2.0           NaN   
3   2020-03-16        Aruba        0.0         NaN          2.0           NaN   
4   2020-03-17        Aruba        1.0         NaN          3.0           NaN   
5   2020-03-18        Aruba        1.0         NaN          4.0           NaN   
6   2020-03-19        Aruba        0.0         NaN          4.0           NaN   
7   2020-03-20        Aruba        1.0         NaN          5.0           NaN   
8   2020-03-21        Aruba        0.0         NaN          5.0           NaN   
9   2020-03-22        Aruba        4.0         NaN          9.0           NaN   
10  2020-03-23        Aruba        0.0         NaN          9.0           NaN   
11  2020-03-24        Aruba 

### Now, we aim to examine the total cases and total deaths per year, along with the fluctuations observed in the most affected countries.

In [50]:
cases = cases.copy()

# Convert 'date' to datetime
cases.loc[:, 'date'] = pd.to_datetime(cases['date'], errors='coerce')

# Check for any non-datetime values and remove them
if cases['date'].isnull().any():
    print("Warning: Some 'date' values could not be converted and will be removed.")
    print(cases[cases['date'].isnull()])  # Print the rows with invalid dates for debugging
    cases = cases.dropna(subset=['date'])

cases.loc[:, 'year'] = cases['date'].apply(lambda x: x.year if pd.notnull(x) else None)

# Group by 'country_name' and year extracted from 'date', then aggregate the maximum values
result = cases.groupby(['country_name', 'year']).agg({
    'total_cases': 'max',
    'total_deaths': 'max'
}).reset_index()

result.rename(columns={'date': 'year'}, inplace=True)

most_affected_by_cases_through_years = result.sort_values(by='total_cases', ascending=False)
most_affected_by_deaths_through_years = result.sort_values(by='total_deaths', ascending=False)

print("Most_affected_by_cases_through_years:")
print(most_affected_by_cases_through_years.head(12))
print("\nMost_affected_by_deaths_through_years:")
print(most_affected_by_deaths_through_years.head(12))

       date                    country_name  new_cases  new_deaths  \
1092    NaT     Africa Eastern And Southern        NaN         NaN   
2203    NaT      Africa Western And Central        NaN         NaN   
5488    NaT                      Arab World        NaN         NaN   
8831    NaT                  American Samoa        NaN         NaN   
20968   NaT                    Bahamas, The        NaN         NaN   
...     ...                             ...        ...         ...   
185559  NaT                         Turkiye        NaN         NaN   
193594  NaT  St. Vincent And The Grenadines        NaN         NaN   
193595  NaT                   Venezuela, Rb        NaN         NaN   
194673  NaT           Virgin Islands (U.S.)        NaN         NaN   
199742  NaT                     Yemen, Rep.        NaN         NaN   

        total_cases  total_deaths country_code  population_2021  
1092            NaN           NaN          AFE      702976832.0  
2203            NaN        

We observe that the United States was the most affected country in terms of both cases and deaths. Let's now determine the year-to-year percentage increase or decrease in these figures for the United States.

In [36]:
us2021 = most_affected_by_cases_through_years[
    (most_affected_by_cases_through_years['country_name'] == 'United States') & 
    (most_affected_by_cases_through_years['year'] == 2021)
]
us2022 = most_affected_by_cases_through_years[
    (most_affected_by_cases_through_years['country_name'] == 'United States') & 
    (most_affected_by_cases_through_years['year'] == 2022)
]
us2023 = most_affected_by_cases_through_years[
    (most_affected_by_cases_through_years['country_name'] == 'United States') & 
    (most_affected_by_cases_through_years['year'] == 2023)
]

us_total_cases_2021 = us2021['total_cases'].values[0]
us_total_cases_2022 = us2022['total_cases'].values[0]
us_total_cases_2023 = us2023['total_cases'].values[0]

# Calculate percentage change from 2021 to 2022
us_cases_percent_change_2021_2022 = round(((us_total_cases_2022 - us_total_cases_2021) / us_total_cases_2021) * 100, 2)
us_cases_percent_change_2022_2023 = round(((us_total_cases_2023 - us_total_cases_2022) / us_total_cases_2022) * 100, 2)

print("Cases percentage change from 2021 to 2022:")
print(us_cases_percent_change_2021_2022,'% ' 'increase')
print("\nCases percentage change from 2022 to 2023:")
print(us_cases_percent_change_2022_2023,'% ' 'increase')

Cases percentage change from 2021 to 2022:
83.52 % increase

Cases percentage change from 2022 to 2023:
3.01 % increase


Let's now do the same for India which is the second most affected country by cases

In [51]:
india2021 = most_affected_by_cases_through_years[
    (most_affected_by_cases_through_years['country_name'] == 'India') & 
    (most_affected_by_cases_through_years['year'] == 2021)
]
india2022 = most_affected_by_cases_through_years[
    (most_affected_by_cases_through_years['country_name'] == 'India') & 
    (most_affected_by_cases_through_years['year'] == 2022)
]
india2023 = most_affected_by_cases_through_years[
    (most_affected_by_cases_through_years['country_name'] == 'India') & 
    (most_affected_by_cases_through_years['year'] == 2023)
]

india_total_cases_2021 = india2021['total_cases'].values[0]
india_total_cases_2022 = india2022['total_cases'].values[0]
india_total_cases_2023 = india2023['total_cases'].values[0]

# Calculate percentage change from 2021 to 2022
india_cases_percent_change_2021_2022 = round(((india_total_cases_2022 - india_total_cases_2021) / india_total_cases_2021) * 100, 2)
india_cases_percent_change_2022_2023 = round(((india_total_cases_2023 - india_total_cases_2022) / india_total_cases_2022) * 100, 2)

print("Cases percentage change from 2021 to 2022:")
print(india_cases_percent_change_2021_2022,'% ' 'increase')
print("\nCases percentage change from 2022 to 2023:")
print(india_cases_percent_change_2022_2023,'% ' 'increase')

Cases percentage change from 2021 to 2022:
28.16 % increase

Cases percentage change from 2022 to 2023:
0.02 % increase


Let's proceed to calculate the percentage increase in deaths for the United States.

In [52]:
us2021d = most_affected_by_deaths_through_years[
    (most_affected_by_deaths_through_years['country_name'] == 'United States') & 
    (most_affected_by_deaths_through_years['year'] == 2021)
]
us2022d = most_affected_by_deaths_through_years[
    (most_affected_by_deaths_through_years['country_name'] == 'United States') & 
    (most_affected_by_deaths_through_years['year'] == 2022)
]
us2023d = most_affected_by_deaths_through_years[
    (most_affected_by_deaths_through_years['country_name'] == 'United States') & 
    (most_affected_by_deaths_through_years['year'] == 2023)
]

us_total_deaths_2021 = us2021d['total_deaths'].values[0]
us_total_deaths_2022 = us2022d['total_deaths'].values[0]
us_total_deaths_2023 = us2023d['total_deaths'].values[0]

# Calculate percentage change from 2021 to 2022
us_deaths_percent_change_2021_2022 = round(((us_total_deaths_2022 - us_total_deaths_2021) / us_total_deaths_2021) * 100, 2)
us_deaths_percent_change_2022_2023 = round(((us_total_deaths_2023 - us_total_deaths_2022) / us_total_deaths_2022) * 100, 2)

print("Deaths percentage change from 2021 to 2022:")
print(us_deaths_percent_change_2021_2022,'% ' 'increase')
print("\nDeaths percentage change from 2022 to 2023:")
print(us_deaths_percent_change_2022_2023,'% ' 'increase')

Deaths percentage change from 2021 to 2022:
32.38 % increase

Deaths percentage change from 2022 to 2023:
2.84 % increase


Quick observations:

1. From 2021 to 2022, there was a substantial increase in both cases and deaths in the most affected countries.
2. From 2022 to 2023, the rate of increase dropped significantly.



## Now, let's examine the coronavirus statistics for Greece during the pandemic period.

In [53]:
cases = cases.copy()

greece_cases = cases[cases['country_name'] == 'Greece']

greece_result = greece_cases.groupby('year').agg({
    'total_cases': 'max',
    'total_deaths': 'max',
    'population_2021': 'max' 
}).reset_index()

greece_impact = greece_cases[['total_cases', 'total_deaths', 'population_2021']].max()
print("\nOverall rate values for Greece:")
print(greece_impact)


Overall rate values for Greece:
total_cases         5548487.0
total_deaths          34779.0
population_2021    10641221.0
dtype: float64


In [54]:
percent_of_population_affected = round(greece_impact['total_cases'] / greece_impact['population_2021'] * 100, 2)
print("\nTotal percent of population affected from coronovirus in Greece:")
print(percent_of_population_affected, "%")

percent_of_population_dead = round(greece_impact['total_deaths'] / greece_impact['population_2021'] * 100, 2)
print("\nTotal percent of population died from coronovirus in Greece:")
print(percent_of_population_dead,"%")


Total percent of population affected from coronovirus in Greece:
52.14 %

Total percent of population died from coronovirus in Greece:
0.33 %


Our analysis of the coronavirus pandemic has revealed several critical insights. The United States was the most affected country in terms of both cases and deaths, exhibiting substantial increases from 2021 to 2022. However, the rate of increase in both cases and deaths significantly dropped from 2022 to 2023.

Globally, the top three countries most affected by coronavirus cases were the United States, India, and France, while the United States, Brazil, and India were most affected in terms of deaths. By calculating the ratios of total cases and deaths to population, we gained a clearer understanding of the pandemic's impact on each country.

For Greece, the data showed notable trends in cases and deaths over the pandemic period, contributing to our comprehensive understanding of how different regions were impacted.

These findings underscore the need for ongoing vigilance and targeted responses to manage future health crises effectively. Future analyses could further refine our conclusions by considering factors such as the quality and strength of healthcare systems, the effectiveness of public health interventions, and the socio-economic impacts of the pandemic.